## Program to read TRIAXYS Wave files and plot results

In [ ]:
using CSV
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk

################################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")
# Pick directory containing Triaxys HNE .txt files
triaxys_directory = pick_folder("C:\\QGHL\\Wave_data\\")

# build list of all Triaxys Wave files in selected directory
triaxys_wave_files = filter(x->length(x)==13,readdir(triaxys_directory))

# Check whether any triaxys files exist in selected directory. If not, EXIT!
if length(triaxys_wave_files) == 0
    println("No Triaxys WAVE files found in "*triaxys_directory)
    exit;
else
    println(string(length(triaxys_wave_files)) * " daily WAVE files found")
end

# create a df that will hold all the WAVE data from the daily files in selected monthly directory
wave_df = DataFrame(Received = [], Year = [], MonthDay = [], Time = [], Buoy_ID = [], Location = [], Zero_Crossings = [], Havg = [],
    Tz = [], Hmax = [], Hsig = [], Tsig = [], H10 = [], T10 = [], TMean = [], Tp = [],Tp5 = [], Hm0 = [],
    Mean_Mag_Dirn = [], Mean_Spread = [], Mean_True_Dirn = [], Te = [], Wave_Steepness = []);

# add each day's wave data to df
for wave_file in triaxys_wave_files

    # get the daily wave data and append to df
    df = DataFrame(CSV.File(triaxys_directory * "//" * wave_file; header=false, skipto=2, delim="\t",ignorerepeated=false));

    colnames = ["Received", "Year", "MonthDay", "Time", "Buoy_ID", "Location", "Zero_Crossings", "Havg", "Tz", "Hmax", "Hsig",
        "Tsig", "H10", "T10", "TMean", "Tp", "Tp5", "Hm0", "Mean_Mag_Dirn", "Mean_Spread", "Mean_True_Dirn", "Te", "Wave_Steepness"]
    rename!(df, Symbol.(colnames))

    append!(wave_df,df);
    
end

# Remove columns that don't contain any QGHL data - this may change in the future!!
select!(wave_df, Not([:Received, :Buoy_ID, :Location, :Mean_True_Dirn]))

# get day and time of start of record
date = []
for row in 1:size(wave_df)[1]
    push!(date,DateTime(wave_df.Year[row],div(wave_df.MonthDay[row],100),mod(wave_df.MonthDay[row],100),
            div(wave_df.Time[row],10000),mod(div(wave_df.Time[row],100),100),mod(wave_df.Time[row],100)))
end

# Insert date column 
insertcols!(wave_df, 1, :Date => date);

first_date = first(date); last_date = last(date);
first_date_string = Dates.format(first_date, "dd/mm/yyyy HH:MM");
last_date_string = Dates.format(last_date, "dd/mm/yyyy HH:MM");
title_string = first_date_string * " to " * last_date_string;

p1 = plot(date,wave_df.Hmax, c=:red, label="Hmax", ylabel="Height (m)", title = title_string, titlefontsize=10)
p1 = plot!(date,wave_df.Hsig, c=:blue, label="Hsig")

p2 = plot(date,wave_df.Tp, c=:red, label="Tp", ylabel="Period (s)")
p2 = plot!(date,wave_df.Tz, c=:blue, label="Tz")

p3 = plot(date,wave_df.Mean_Mag_Dirn, c=:red, label="Mean magnetic dirn.", ylim=(0,360), 
    yflip=true, yticks = 0:30:360, ylabel="Dirn. (" * L"^o" * "Magnetic)")

# get number of days between start and end of record
day_diff = (Dates.DateTime(last_date) - Dates.DateTime(first_date)) / Millisecond(1000) / 86400

if day_diff < 31
    
    tm_tick = range(first_date,last_date,step=Day(1))
    ticks = Dates.format.(tm_tick,"dd")

elseif day_diff < 90
    
    tm_tick = range(first_date,last_date,step=Day(2))
    ticks = Dates.format.(tm_tick,"dd/mm")
    
else
    
    tm_tick = range(first_date,last_date,step=Month(1))
    ticks = Dates.format.(tm_tick,"mm")
    
end

wave_plot = plot(p1, p2, p3, layout = (3, 1), size = (1400, 600),
    xlim=(first_date,last_date),  xticks=(tm_tick,ticks), xtickfontsize=7, 
    ytickfontsize=8,
    framestyle = :box,fg_legend=:transparent, legend=:topright,
    topmargin=1Plots.mm, leftmargin=8Plots.mm,grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)            

display(wave_plot)

################################################
################################################
################################################
##           END OF MAIN PROGRAM
################################################
################################################
################################################


In [ ]:
show(wave_df,allcols=true)

## Plot Wave Height, Period, and Direction for particular day

In [ ]:
# Determine first and last record times
first_day = floor(first(wave_df.Date), Dates.Day(1))
last_day = floor(last(wave_df.Date), Dates.Day(1))

available_dates = collect(first_day:Day(1):last_day)
dates = Dates.format.(available_dates, "yyyy-mm-dd")

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global date_choice = get_value(lb);
    
    global date_start = DateTime(date_choice[1], dateformat"y-m-d H:M")
    global date_end = date_start + Day.(1)

    println(date_start,' ',date_end)
    flush(stdout)
    global record_df = wave_df[date_start .<= wave_df.Date .< date_end, :]
    
    # Check whether data is available for selected 30-minute record
    if !isempty(record_df)
    
        title_string = Dates.format(date_start, "dd/mm/yyyy");

        # Wave heights
        p1 = plot(record_df.Date,record_df.Hmax, c=:red, fillrange = 0, fillalpha = 0.025, fillcolor = :red, 
                lw=2, label="Hmax", ylabel="Height (m)", title = title_string, titlefontsize=10)
        p1 = plot!(record_df.Date,record_df.Hsig, fillrange = 0, fillalpha = 0.025, fillcolor = :blue, c=:blue, lw=2, label="Hsig")

        # Wave Periods
        p2 = plot(record_df.Date,record_df.Tp, fillrange = 0, fillalpha = 0.025, fillcolor = :red, c=:red, lw=2, label="Tp", ylabel="Period (s)")
        p2 = plot!(record_df.Date,record_df.Tz, fillrange = 0, fillalpha = 0.025, fillcolor = :blue, c=:blue, lw=2, label="Tz")
 
        # Wave Direction and Spread
        p3 = plot(record_df.Date, [record_df.Mean_Mag_Dirn record_df.Mean_Mag_Dirn], ylim=(0,360), yflip=true, yticks = 0:45:360, 
                fillrange=[record_df.Mean_Mag_Dirn .- record_df.Mean_Spread record_df.Mean_Mag_Dirn .+ record_df.Mean_Spread], 
                lw=2, fillalpha=0.3, c=[:pink :lightblue], label=["" ""], ylabel="Dirn. (" * L"^o" * "Magnetic)")
        p3 = plot!(record_df.Date, record_df.Mean_Mag_Dirn, lw=2, c=:red, label = "Mean Magnetic Dirn.")

        tm_tick = range(date_start,date_end,step=Hour(1))
        ticks = Dates.format.(tm_tick,"HH:MM")

        wave_plot = plot(p1, p2, p3, layout = (3, 1), size = (1400, 800),
            xlim=(date_start,date_end),  xticks=(tm_tick,ticks), xtickfontsize=7, 
            ytickfontsize=8,
            framestyle = :box,fg_legend=:transparent, legend=:topright,
            topmargin=1Plots.mm, leftmargin=8Plots.mm,grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)            

        display(wave_plot)
    
    else
        
        println("Alert: No data available between " * Dates.format(date_start, "dd/mm/yyyy HH:MM") * " and " * Dates.format(date_end, "dd/mm/yyyy HH:MM"))
        flush(stdout)
        
    end

end

In [ ]:
date_choice